# Prep imports and libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import quantile_transform
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import OneHotEncoder  ##. better to use dummy from pandas 
from sklearn.preprocessing import PowerTransformer
from scipy.stats import boxcox
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from scipy.stats import boxcox
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
pd.options.display.max_rows = 50
import qgrid
import glob
import math
import os

# Load demographic and nutrition info

In [181]:
def load_csv_dataframes(path):
    '''Loads all csv files in a folder and stores them in a list of dataframes.
    Returns list of dataframes.
    [attributes]
    path: path of folder with csvs inside
    '''
    all_files = glob.glob(path + "/*.csv")
    dataframes_list=[]
    file_count=len(all_files)
    for i in range(file_count):
        temp_df = pd.read_csv(all_files[i])
        dataframes_list.append(temp_df)

    return dataframes_list

df_list=load_csv_dataframes(r'C:\Users\MichaelTaylo_c9zoof3\Documents\GitHub\IH_Berliners\Data\Files to merge')

# Make all country names lowercase

In [182]:
def string_lowercase(df):
    '''Makes all string values in dataframe lowercase.
    Returns updated dataframe.
    [attributes]
    df: dataframe to apply lower on'''
    df=df.applymap(lambda x:x.lower() if type(x) == str else x)
    return df

for i in df_list:
    i.update(string_lowercase(i))

# Merge files

In [183]:
df=pd.merge(df_list[0], df_list[1], on='country')


In [184]:
qgrid.show_grid(df,grid_options={'forceFitColumns': False})

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

# Create mortality rate as dependent value

In [185]:
df['mortality_rate_covid']=(df['deaths']/df['confirmed'])*100
df[['mortality_rate_covid']]

,mortality_rate_covid
0,4.352069
1,1.717095
2,2.677922
3,2.367660
4,2.430556
...,...
143,0.785628
144,0.000000
145,1.763224
146,1.365849


# Check dataset as is

In [186]:
qgrid.show_grid(df,grid_options={'forceFitColumns': False})

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

# Gather count of NANs by column

In [3]:
count_of_na=df.isna().sum()
qgrid.show_grid(count_of_na,grid_options={'forceFitColumns': False})


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

## Get a list of all columns with more than 10 NANs and drop them from dataset

In [7]:
large_na=count_of_na[count_of_na>20]
len(large_na)

26

In [8]:
for i in large_na.index:
    df.drop(columns=i,inplace=True)

## Output a file at this stage

In [9]:
output_path=r'C:\Users\MichaelTaylo_c9zoof3\Documents\GitHub\IH_Berliners\Data\2. After dropped columns\combined_data_without_dropped_columns.csv'
df.to_csv(output_path, index=False)

## Check for columns with few NANs and export it as a file to clean manually

In [10]:
count_of_na=df.isna().sum()
small_na=count_of_na[count_of_na>5]
df_nas=df[small_na.index]

In [11]:
df_nas['country']=df['country']
df_nas

C:\Users\MichaelTaylo_c9zoof3\AppData\Local\Temp\ipykernel_8976\965910400.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nas['country']=df['country']


,"gni per capita, atlas method (current us$)","immunization, hepb3 (% of one-year-old children)",improved sanitation facilities (% of population with access),"improved sanitation facilities, rural (% of rural population with access)","improved sanitation facilities, urban (% of urban population with access)",improved water source (% of population with access),"improved water source, rural (% of rural population with access)",maternal leave benefits (% of wages paid in covered period),number of weeks of maternity leave,people practicing open defecation (% of population),"people practicing open defecation, rural (% of rural population)","people practicing open defecation, urban (% of urban population)",undernourished,country
0,630.0,78.0,31.9,27.0,45.1,55.3,47.0,100.0,90.0,12.7,17.4,0.0,29.8,afghanistan
1,4290.0,98.0,93.2,90.2,95.5,95.1,95.2,65.0,365.0,0.0,0.0,0.0,6.2,albania
2,4870.0,95.0,87.6,82.2,89.8,83.6,81.8,100.0,98.0,0.8,2.3,0.2,3.9,algeria
3,4180.0,64.0,51.6,22.5,88.6,49.0,28.2,100.0,90.0,30.3,53.8,0.5,25.0,angola
4,13390.0,99.0,NaN,NaN,NaN,97.9,97.9,78.0,91.0,NaN,NaN,NaN,NaN,antigua and barbuda
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,NaN,87.0,94.4,69.9,97.5,93.1,77.9,100.0,182.0,5.4,28.6,2.5,21.2,venezuela
159,1980.0,97.0,78.0,69.7,94.4,97.6,96.9,100.0,180.0,0.7,1.1,0.0,9.3,vietnam
160,NaN,69.0,NaN,NaN,NaN,NaN,NaN,100.0,70.0,NaN,NaN,NaN,38.9,yemen
161,1500.0,90.0,43.9,35.7,55.6,65.4,51.3,100.0,84.0,13.7,22.3,1.3,46.7,zambia


In [12]:
output_path=r'C:\Users\MichaelTaylo_c9zoof3\Documents\GitHub\IH_Berliners\Data\few_nans_to_manually_fix.csv'
df_nas.to_csv(output_path, index=False)

### Make copy of df to ensure nothing goes wrong in merging data back in

In [13]:
df_latest=df.copy()

In [14]:
df_no_nan=pd.read_csv(r"C:\Users\MichaelTaylo_c9zoof3\Documents\GitHub\personal\People Analytics\Data\Final\few_nans_to_manually_fix_done.csv")

In [15]:
df_latest.update(df_no_nan,overwrite=False)

In [204]:
df_latest=to_csv(r"C:\Users\MichaelTaylo_c9zoof3\Documents\GitHub\IH_Berliners\Data\3. Full merge no nan\full_merge_no_nan.csv")

In [16]:
df_latest.isna().sum()

country                                               0
age dependency ratio (% of working-age population)    1
age dependency ratio, old                             1
age dependency ratio, young                           1
age population, age 0, female, interpolated           1
                                                     ..
protein_treenuts                                      0
protein_vegetal products                              0
protein_vegetable oils                                0
protein_vegetables                                    0
protein_miscellaneous                                 0
Length: 251, dtype: int64

In [206]:
df=pd.read_csv(r"C:\Users\MichaelTaylo_c9zoof3\Documents\GitHub\IH_Berliners\Data\3. Full merge no nan\full_merge_no_nan.csv")

In [17]:
def replace_nan_integer_with_mean(df,integers_with_nan):
    #Fill missing numbers with the average of the column
    for j in integers_with_nan:
        temp_mean=np.mean(df[j])
        df[j]=df[j].fillna(temp_mean)
    return df

def get_columns_with_nan(df):
    # This creates a list ## integers_with_nan ## which is a list of numerical columns with blanks in them.  
    col_list=[]
    col_list=df.columns
    integers_with_nan=[]
    for i in col_list:
        if df[i].isnull().values.any()==True:
            if df[i].dtype!='O':
                integers_with_nan.append(i)
    return integers_with_nan

integers_with_nan=get_columns_with_nan(df)
#integers_with_nan=[list]


#Replace missing numbers with mean
df=replace_nan_integer_with_mean(df,integers_with_nan)

In [18]:
df.to_csv(r"C:\Users\MichaelTaylo_c9zoof3\Documents\GitHub\IH_Berliners\Data\3. Full merge no nan\full_merge_no_nans2.csv",index=False)